In [16]:
#!import ./setup-clients.ipynb

const string PostsIndex = "posts";

## Srolling - `Elastic.Clients.Elasticsearch`

<https://www.elastic.co/guide/en/elasticsearch/reference/current/paginate-search-results.html#scroll-search-results>


In [31]:
#nullable enable
using Elastic.Clients.Elasticsearch;

public class ScrollConfiguration
{
    public int PageSize { get; set; } = 250;

    public Elastic.Clients.Elasticsearch.Duration KeepAlive { get; set; } = new Elastic.Clients.Elasticsearch.Duration("1m");

    public bool DebugOutput { get; set; } = false;

    public static ScrollConfiguration Default => new ();
}

public static class ElasticsearchClientExtensions
{
    public static async IAsyncEnumerable<TDocument> ScrollAsync<TDocument>(
        ElasticsearchClient client,
        Elastic.Clients.Elasticsearch.Indices indices,
        Action<SearchRequestDescriptor<TDocument>>? configure = default, 
        ScrollConfiguration? scrollConfiguration = default,
        [System.Runtime.CompilerServices.EnumeratorCancellation] System.Threading.CancellationToken cancellationToken = default)
    {
        Elastic.Clients.Elasticsearch.OpenPointInTimeResponse? pit = default;

        scrollConfiguration ??= ScrollConfiguration.Default;

        try
        {
            pit = await client.OpenPointInTimeAsync(
                indices,
                d => d.KeepAlive(scrollConfiguration!.KeepAlive),
                cancellationToken
            );

            EnsureResponseHasNoErrors(pit);

            var hasDocumentsToSearch = false;
            ICollection<FieldValue>? searchAfter = null;

            do
            {
                var searchResponse = await client.SearchAsync<TDocument>(
                    s =>
                    {
                        ConfigureSearch(s);

                        if (searchAfter is not null)
                        {
                            s.SearchAfter(searchAfter);
                        }
                    },
                    cancellationToken
                );

                EnsureResponseHasNoErrors(searchResponse);

                foreach (var document in searchResponse.Documents)
                {
                    yield return document;
                }

                searchAfter = searchResponse.Hits.LastOrDefault()?.Sort?.ToArray();
                hasDocumentsToSearch = searchAfter is { Count: > 0 };
            } while (hasDocumentsToSearch);
        }
        finally
        {
            if (pit is not null)
            {
                var closeResponse = await client.ClosePointInTimeAsync(
                    d => d.Id(pit.Id),
                    cancellationToken
                );
                EnsureResponseHasNoErrors(closeResponse);
            }
        }

        SearchRequestDescriptor<TDocument> ConfigureSearch(SearchRequestDescriptor<TDocument> descriptor)
        {
            descriptor = descriptor
                .Index(indices)
                .Pit(pit!.Id, d => d.KeepAlive(scrollConfiguration!.KeepAlive))
                .Size(scrollConfiguration.PageSize);

            if(configure is not null)
            {
                configure(descriptor);
            }
            else
            {
                descriptor.Sort(
                    [
                        SortOptions.Field(
                            "_doc",
                            new Elastic.Clients.Elasticsearch.FieldSort { Order = Elastic.Clients.Elasticsearch.SortOrder.Asc })
                    ]);
            }

            return descriptor;
        }

        void EnsureResponseHasNoErrors(ElasticsearchResponse response)
        {
            if (!response.IsSuccess() && response.TryGetOriginalException(out var ex))
            {
                throw ex!;
            }
        }
    }
}

In [32]:
var scrollConfiguration = ScrollConfiguration.Default;
scrollConfiguration.PageSize = 500;
scrollConfiguration.DebugOutput = false;

var source = ElasticsearchClientExtensions.ScrollAsync<BlogPost>(client, PostsIndex, scrollConfiguration: scrollConfiguration);

display(await source.CountAsync());
// var posts = await source.ToListAsync();